In [34]:
from langchain_core.tools import BaseTool, StructuredTool, tool
from langchain_core.prompts import PromptTemplate
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser

In [ ]:
@tool
def GetFlightStatus(flight_number: str) -> str:
    """Get the status of a flight and schedule."""
    # We can invoke real API from here to get flight status.
    # In a real implementation, this would call an external API.
    # Here, we'll just return a mock response.
    #mock_status = {
    #    "AA100": "On Time",
    #    "DL200": "Delayed",
    #    "UA300": "Cancelled"
    #}
    
    #return mock_status.get(flight_number, "Flight not found")
    return f"Flight {flight_number} departed at 5:20 PM. It is on-time and expected to arrive at 8:10PM at Gate B12"  # Mock response

In [36]:
print(GetFlightStatus.name)
print(GetFlightStatus.description)
print(GetFlightStatus.args)

GetFlightStatus
Get the status of a flight and schedule.
{'flight_number': {'title': 'Flight Number', 'type': 'string'}}


In [37]:
GetFlightStatus

StructuredTool(name='GetFlightStatus', description='Get the status of a flight and schedule.', args_schema=<class 'langchain_core.utils.pydantic.GetFlightStatus'>, func=<function GetFlightStatus at 0x111b2e0c0>)

In [38]:
prompt = PromptTemplate.from_template(
    """Based on the context: {context}

Answer the query: {query} about flight: {flight}

Rules:
- Answer ONLY if you find flight details in the context
- If context says "Flight not found", respond with "Flight not found"
- Use one or two words maximum
- Do not make up information

Answer:"""
)

In [39]:
llm = OllamaLLM(model="llama3.2", temperature=0)

In [40]:
flight="IE210"

In [41]:
context = GetFlightStatus.run(flight)
context

'Flight IE210 departed at 5:20 PM. It is on-time and expected to arrive at 8:10PM at Gate B'

In [42]:
output_parser = StrOutputParser()

In [43]:
chain = prompt | llm | output_parser

In [44]:
print ("flight number", flight)
chain.invoke({
    "context": context,
    "query": "status",
    "flight": flight
})

flight number IE210


'On-time.'

In [45]:
chain.invoke({
    "context": context,
    "query": "departure time",
    "flight": flight
})

'5:20 PM'

In [46]:
chain.invoke({
    "context": context,
    "query": "gate number",
    "flight": flight
})

'B'